# Spark Learning Note - MLlib
Jia Geng | gjia0214@gmail.com


*Do not run multiple jupyter notebooks at the same time. It sometime messes up with the java gate way and cause error for returning the performance metrics*

<a id='directory'></a>

## Directory

- [Data Source](https://github.com/databricks/Spark-The-Definitive-Guide/tree/master/data/)
- [1. Regression Models in MLlib](#sec1)
- [2. Linear Regression](#sec2)
- [3. Generalized Linear Regression](#sec3)
- [4. Tree-based Algorithms](#sec4)
- [5. Advanced Methods](#sec5)
    - [5.1  Decision Tree](#sec5-1)
    - [5.2 Random Forest & Gradient-boosted Tree](#sec5-2)
- [6. Evaluators for Classification and Automating Model Tuning](#sec6)

In [24]:
# check java version 
# use sudo update-alternatives --config java to switch java version if needed.
!java -version

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1ubuntu1-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [25]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName('MLexample').getOrCreate()
spark

In [26]:
data_path = '/home/jgeng/Documents/Git/SparkLearning/book_data/regression' 
data = spark.read.parquet(data_path)
data.printSchema()
data.show()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)

+--------------+-----+
|      features|label|
+--------------+-----+
|[3.0,10.1,3.0]|  2.0|
| [2.0,1.1,1.0]|  1.0|
|[1.0,0.1,-1.0]|  0.0|
|[1.0,0.1,-1.0]|  0.0|
| [2.0,4.1,1.0]|  2.0|
+--------------+-----+



## 1. Regression Models in MLlib <a id='sec1'></a>

Spark supports the following algorithms:
- Linear Regression
- Generalized Linear Model
- Isotonic Regression
- Regression Tree
- Random Forest with Regression Tree
- Gradient-boosted Trees
- Survival Regression

[back to top](#directory)

## 2. Linear Regression <a id='sec2'></a>

Share the same params as the logistic regression.

**Model Summary**

**There are a bunch of metrics that supposely supported by pyspark. However, it seems that metrics such as pValues are broken right now, even the solver was set to normal.**

[back to top](#directory)

In [27]:
from pyspark.ml.regression import LinearRegression
print(LinearRegression().explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model.

In [28]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression().setMaxIter(10).setElasticNetParam(0.8).setRegParam(0.3)\
                        .setSolver('normal')
lrclf = lr.fit(data)

## 3. Generalized Linear Regression <a id='sec3'></a>

For more generalized linear regression. You can select differnet noise distribution, response type and there are also different kinds of link functions that maps the input to the mean of the distribution funcion. 

For spark 2.2, the gml can only accept less than 4096 features.

|Family|Response Type|Suppored Links|
|-|-|-|
|Gaussian|Continuous|Identity, Log, Inverse|
|Binomial|Binary|Logit, Probi, CLogLog|
|Possian|Count|Log, Identity, Sqrt|
|Gamma|Continuous|Inverse, Identity, Log|
|Tweedue|Zero-inflated Continuous|Power Link Function|

[back to top](#directory)

In [29]:
from pyspark.ml.regression import GeneralizedLinearRegression

print(GeneralizedLinearRegression().explainParams())

family: The name of family which is a description of the error distribution to be used in the model. Supported options: gaussian (default), binomial, poisson, gamma and tweedie. (default: gaussian)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
link: The name of link function which provides the relationship between the linear predictor and the mean of the distribution function. Supported options: identity, log, inverse, logit, probit, cloglog and sqrt. (undefined)
linkPower: The index in the power link function. Only applicable to the Tweedie family. (undefined)
linkPredictionCol: link prediction (linear predictor) column name (undefined)
maxIter: max number of iterations (>= 0). (default: 25)
offsetCol: The offset column name. If this is not set or empty, we treat all instance offsets as 0.0 (undefined)
predictionCol: prediction column name. (default: prediction)
regPa

## 4. Tree-based Algorithms <a id='sec4'></a>

APIs are similar with classification, except for the loss options etc.

[back to top](#directory)

In [30]:
from pyspark.ml.regression import GBTRegressor, RandomForestRegressor

print(GBTRegressor().explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [31]:
print(RandomForestRegressor().explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

## 5. Advanced Methods <a id='sec5'></a>

[back to top](#directory)

### 5.1 Survival Regression <a id='sec5-1'></a>

Spark has implementation of of **Accelerated Failure Time** model, which describe the log of the survival time. *https://onlinelibrary.wiley.com/doi/abs/10.1002/sim.4780111409*. 

A more well-known model **Cox Proportional Hazard's** model does not scale well so it is not supported by spark.

An example:
https://spark.apache.org/docs/latest/api/R/spark.survreg.html

[back to top](#directory)

### 5.2 Isotonic Regression <a id='sec5-2'></a>

Isotonic regression specify **a piecewise linear function that is always monotonically increasing**. It can not decrease.

https://spark.apache.org/docs/2.2.0/mllib-isotonic-regression.html

[back to top](#directory)

# 6. Evaluators and Automatic Model Tuning <a id='sec6'></a>

`RegressionEvaluator` provides the functionalities for regression model evaluation.
- The `RegressionEvaluator` takes the prediction conlumn and the true label comlumn as input. - It supports metrics such as rmse, mse, r2 and mae.
- It can be used to build up pipline for automatic tuning.


To build a automatic tuning pipeline, we need
- A ml algorithm class
- A transformer (if needed)
- A `Pipeline` that takes in the transformer and ml algorithm object
- A `Evaluator` that provide performance metrics for model selection
- A `ParamGridBuilder` that takes in the candidate params for grid search
- A Validator (`CrossValidator`, `TrainValidationSplit`) that takes in the Pipeline and the `ParamGridBuilder`

The basic work flow is:
- Step1: Initiate the transformer, the model builder and build the pipeline
- Step2: Set up the evaluator
- Step3: Set up the grid search
- Step4: Build the validator
- Step5: fit the validator

[back to top](#directory)

In [32]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# Step 1
# a. build the model, specifying the fixed params
glr = LinearRegression()
# b. build the pipline, here, no transformation stage
pipline = Pipeline().setStages([glr])

# Step 2 - build the evaluator
evaluator = RegressionEvaluator().setMetricName('rmse')\
                                .setPredictionCol('prediction')\
                                .setLabelCol('label')

In [33]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Step 3 - Set up grid search
params = ParamGridBuilder().addGrid(glr.regParam, [0, 0.5, 1]).build()

# Step 4 - Set up the validator
cv = CrossValidator()\
                    .setEstimator(pipline)\
                    .setEstimatorParamMaps(params)\
                    .setEvaluator(evaluator)\
                    .setNumFolds(2)

# Step 5 - fit!
model = cv.fit(data)

In [34]:
from pyspark.mllib.evaluation import RegressionMetrics  # under the mllib not ml!!!!

# functions under mllib are low level functions
# RegressionMetrics works on RDDs instead of the Dataframes

# well this should be on testing data...
# for this example, use whole data for demonstration
# model. will return the best model
out = model.transform(data).select('prediction', 'label')\
            .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = RegressionMetrics(out)
print('mse', metrics.meanSquaredError)
print('r2', metrics.r2)
print('rmse', metrics.rootMeanSquaredError)
print('ev', metrics.explainedVariance)

mse 0.21432448963739983
r2 0.7320943879532502
rmse 0.4629519301584127
ev 0.32923447021104896
